# Preliminary Model Build for Mockup

In [1]:
# Import Warnings and Dependencies
import warnings
warnings.filterwarnings('ignore')
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib
from pathlib import Path
from collections import Counter



In [2]:
# import sklearn functions as needed 2022.11.1 Load then remove
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

### Data to Load

In [3]:
# Load in Data
file_path = Path("..\Data\Model_Data\combined_data.csv") 
# Pulled in cleaned combined_data from SQL database directly
housing_data_df = pd.read_csv(file_path)
housing_data_df


,region_id,region_desc,week,sales_inventory,civilian_labor_force,employment,unemployment,unemployment_rate,med_house_price,rent_index,sales_count
0,394463,"Chicago, IL",3/31/2018,38581,4963614.0,4753730.0,209884.0,4.2,284600,1610.46,9053
1,394463,"Chicago, IL",4/30/2018,42253,4960889.0,4756797.0,204092.0,4.1,294600,1622.52,11208
2,394463,"Chicago, IL",5/31/2018,45757,4959982.0,4759532.0,200450.0,4.0,300600,1634.70,13708
3,394463,"Chicago, IL",6/30/2018,47492,4961285.0,4762416.0,198869.0,4.0,302267,1645.26,15524
4,394463,"Chicago, IL",7/31/2018,48984,4964193.0,4765720.0,198473.0,4.0,301967,1651.54,14358
...,...,...,...,...,...,...,...,...,...,...,...
270,394902,"Nashville, TN",5/31/2022,4590,1138543.0,1107909.0,30634.0,2.7,479967,1860.25,3382
271,394902,"Nashville, TN",6/30/2022,5436,1139523.0,1108733.0,30790.0,2.7,494966,1884.61,3700
272,394902,"Nashville, TN",7/31/2022,6341,1137422.0,1106593.0,30829.0,2.7,498300,1910.61,3041
273,394902,"Nashville, TN",8/31/2022,7106,1134574.0,1103911.0,30663.0,2.7,499633,1923.84,2859


In [4]:
# Check data types
housing_data_df.dtypes

region_id                 int64
region_desc              object
week                     object
sales_inventory           int64
civilian_labor_force    float64
employment              float64
unemployment            float64
unemployment_rate       float64
med_house_price           int64
rent_index              float64
sales_count               int64
dtype: object

In [ ]:
# Need to remove latest data with 9/30/2022 since unemployment wasn't included
#housing_data_df = housing_data_df.loc[(housing_data_df["week"] == "9/30/2022")]
#housing_data_df

In [27]:
#housing_data_cleaned = housing_data_df.drop(housing_data_df.index[[54,109,164,219,274]])
housing_data_cleaned = housing_data_df.dropna()
housing_data_cleaned.shape
housing_data_cleaned

,region_id,region_desc,week,sales_inventory,civilian_labor_force,employment,unemployment,unemployment_rate,med_house_price,rent_index,sales_count
0,394463,"Chicago, IL",3/31/2018,38581,4963614.0,4753730.0,209884.0,4.2,284600,1610.46,9053
1,394463,"Chicago, IL",4/30/2018,42253,4960889.0,4756797.0,204092.0,4.1,294600,1622.52,11208
2,394463,"Chicago, IL",5/31/2018,45757,4959982.0,4759532.0,200450.0,4.0,300600,1634.70,13708
3,394463,"Chicago, IL",6/30/2018,47492,4961285.0,4762416.0,198869.0,4.0,302267,1645.26,15524
4,394463,"Chicago, IL",7/31/2018,48984,4964193.0,4765720.0,198473.0,4.0,301967,1651.54,14358
...,...,...,...,...,...,...,...,...,...,...,...
269,394902,"Nashville, TN",4/30/2022,3895,1134591.0,1104357.0,30234.0,2.7,458300,1840.38,3253
270,394902,"Nashville, TN",5/31/2022,4590,1138543.0,1107909.0,30634.0,2.7,479967,1860.25,3382
271,394902,"Nashville, TN",6/30/2022,5436,1139523.0,1108733.0,30790.0,2.7,494966,1884.61,3700
272,394902,"Nashville, TN",7/31/2022,6341,1137422.0,1106593.0,30829.0,2.7,498300,1910.61,3041


### Split the Data Into Training and Testing

In [28]:
# Create our features
y = housing_data_cleaned["med_house_price"]
X = housing_data_cleaned.drop(columns=["med_house_price","region_desc","week"]) # TBD based on factors to pull in
X.head(10) #Confirm Out has med_house_price dropped? =Yes

,region_id,sales_inventory,civilian_labor_force,employment,unemployment,unemployment_rate,rent_index,sales_count
0,394463,38581,4963614.0,4753730.0,209884.0,4.2,1610.46,9053
1,394463,42253,4960889.0,4756797.0,204092.0,4.1,1622.52,11208
2,394463,45757,4959982.0,4759532.0,200450.0,4.0,1634.70,13708
3,394463,47492,4961285.0,4762416.0,198869.0,4.0,1645.26,15524
4,394463,48984,4964193.0,4765720.0,198473.0,4.0,1651.54,14358
5,394463,49782,4969483.0,4770051.0,199432.0,4.0,1650.94,13634
6,394463,49630,4976756.0,4775171.0,201585.0,4.1,1643.64,10809
7,394463,48916,4984436.0,4779818.0,204618.0,4.1,1632.04,11088
8,394463,46495,4992719.0,4784079.0,208640.0,4.2,1623.91,9566
9,394463,42304,5000345.0,4787426.0,212919.0,4.3,1618.47,8519


In [29]:
# Import sklearn model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
# Create the scaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
#Fitting the scaler
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Determine the Model Type
Will need to evaluate the type based on accuracy and balance reports

Naive Random Oversampling

In [30]:
# Resample the training data with the RandomOversampler
# Implement Random Oversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({363743: 5,
         289316: 5,
         298998: 5,
         327233: 5,
         327392: 5,
         307948: 5,
         324000: 5,
         314308: 5,
         297933: 5,
         332667: 5,
         408522: 5,
         311667: 5,
         363000: 5,
         309967: 5,
         348333: 5,
         328300: 5,
         413423: 5,
         308950: 5,
         316633: 5,
         287666: 5,
         368078: 5,
         308448: 5,
         322467: 5,
         351633: 5,
         416300: 5,
         367333: 5,
         329300: 5,
         368662: 5,
         299933: 5,
         300600: 5,
         283333: 5,
         345000: 5,
         321058: 5,
         318000: 5,
         333333: 5,
         297667: 5,
         319583: 5,
         326633: 5,
         331433: 5,
         287650: 5,
         297967: 5,
         307667: 5,
         503283: 5,
         505967: 5,
         319450: 5,
         364483: 5,
         423150: 5,
         299633: 5,
         401661: 5,
         289965: 5,


In [31]:
# Train the Logistic Regression model using the resampled data
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
# Fit the model
classifier.fit(X_resampled,y_resampled)

LinearRegression()

In [33]:
# Calculated the balanced accuracy score
y_pred = classifier.predict(X_test_scaled)

from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.6584614469360068

Model 2: SMOTE Oversampling

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LinearRegression()
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test,y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred)

In [ ]:

# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

### Additional Tests will be summarized to find the highest accuracy